In [1]:
from data.config import data_config
import pandas as pd
import numpy as np
import shap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse, make_scorer
from sklearn.model_selection import KFold, ParameterSampler
from xgboost.sklearn import XGBRegressor
from datamanager import DataManager
from lightgbm import LGBMRegressor
from tqdm import tqdm
pd.options.display.float_format = '{:.2f}'.format


c:\ProgramData\anaconda3\envs\emt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
target_col = "pm"

dm_w_lags = DataManager(data_config=data_config, target_col=target_col)
dm_w_lags.featurize()

In [3]:
dm_w_lags.df

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,profile_id,...,S_el_lag_1080,S_el_lag_1200,P_el_lag_840,P_el_lag_960,P_el_lag_1080,P_el_lag_1200,pm_lag_840,pm_lag_960,pm_lag_1080,pm_lag_1200
0,19.85,18.81,-0.35,-0.45,0.00,0.19,0.00,0.00,24.55,17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,19.85,18.82,-0.31,-0.33,0.00,0.25,0.00,-0.00,24.54,17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,19.85,18.83,-0.37,-0.44,0.00,0.18,0.00,0.00,24.54,17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,19.85,18.84,-0.32,-0.33,0.01,0.24,0.00,0.00,24.55,17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,19.85,18.86,-0.33,-0.47,0.00,0.21,-0.06,0.04,24.57,17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330811,23.99,30.72,0.79,-2.15,0.00,-0.00,-2.00,1.10,62.15,71,...,5.83,7.08,21238.40,0.32,-4.42,-6.93,65.89,66.32,67.29,67.94
1330812,23.97,30.72,0.78,-2.26,0.00,-0.00,-2.00,1.10,62.14,71,...,12.38,7.27,19963.07,0.54,1.97,-7.10,66.02,66.32,67.29,67.93
1330813,23.98,30.72,0.80,-2.13,0.00,-0.00,-2.00,1.10,62.14,71,...,24.36,7.11,19221.32,0.32,-7.20,-6.95,66.02,66.29,67.28,67.93
1330814,24.00,30.72,0.76,-2.27,0.01,-0.00,-2.00,1.10,62.13,71,...,92.67,7.29,19324.47,0.55,-84.76,-7.11,66.16,66.27,67.27,67.93


In [4]:
print(dm_w_lags.df.columns)

Index(['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'torque', 'i_d',
       'i_q', 'pm', 'profile_id', 'i_s', 'u_s', 'S_el', 'P_el', 'ambient_p10',
       'ambient_p90', 'ambient_rolling_mean', 'coolant_p10', 'coolant_p90',
       'coolant_rolling_mean', 'u_d_p10', 'u_d_p90', 'u_d_rolling_mean',
       'u_q_p10', 'u_q_p90', 'u_q_rolling_mean', 'motor_speed_p10',
       'motor_speed_p90', 'motor_speed_rolling_mean', 'torque_p10',
       'torque_p90', 'torque_rolling_mean', 'i_d_p10', 'i_d_p90',
       'i_d_rolling_mean', 'i_q_p10', 'i_q_p90', 'i_q_rolling_mean',
       'ambient_rolling_std', 'coolant_rolling_std', 'u_d_rolling_std',
       'u_q_rolling_std', 'motor_speed_rolling_std', 'torque_rolling_std',
       'i_d_rolling_std', 'i_q_rolling_std', 'ambient_lag_840',
       'ambient_lag_960', 'ambient_lag_1080', 'ambient_lag_1200',
       'coolant_lag_840', 'coolant_lag_960', 'coolant_lag_1080',
       'coolant_lag_1200', 'u_d_lag_840', 'u_d_lag_960', 'u_d_lag_1080',
       'u_d